# Overview

This notebook demostrates the usage of `piper_control`, our lightweight wrapper
of `piper_sdk` for controlling AgileX Piper robots.

The `piper_sdk` API is powerful and quickly maturing, but it's a bit complex and
under-documented, and we found it helpful to define a simple abstraction for
basic I/O.

There are also several sharp bits in `piper_sdk` which can make the robots seem
tempermental, e.g. becoming unresponsive despite repeated calls to
`MotionCtrl_2`, `EnableArm`, `GripperCtrl`, etc. We've bundled our solutions
into `PiperControl` so `reset` and the various move commands perform as one
would expect.

In [ ]:
# Enable autoreload (optional, use if modifying piper_control.py)
%load_ext autoreload
%autoreload 2

# CAN Connect
This section demonstrates the use of `piper_connect` to establish the CAN
connection to the robot(s).

It is a python counterpart to AgileX's `can_activate.sh`.

Make sure to generate the udev rules as described in the
[Generating udev rules](https://github.com/Reimagine-Robotics/piper_control?tab=readme-ov-file#generating-udev-rules-for-can-adapters) section of the README.

## Poll and Connect

In [ ]:
from piper_control import piper_connect

In [ ]:
ports = piper_connect.find_ports()
print(ports)

In [ ]:
piper_connect.activate()
print(piper_connect.active_ports())

# Initialize and Query State

In [ ]:
from piper_control import piper_control
from piper_control import piper_init
from piper_control import piper_interface

robot = piper_interface.PiperInterface(can_port="can0")

# Resets the robot and enables the motors and motion controller for the arm.
# This call is necessary to be able to both query state and send commands to the
# robot.
piper_init.reset_arm(
    robot,
    arm_controller=piper_interface.ArmController.POSITION_VELOCITY,
    move_mode=piper_interface.MoveMode.JOINT,
)
piper_init.reset_gripper(robot)

print(robot.get_joint_positions())
print(robot.get_joint_velocities())
print(robot.get_joint_efforts())
print(robot.get_gripper_state())

In [ ]:
# Prints out a higher level status of the robot.
robot.show_status()

## Resetting and enabling the robot

In [ ]:
# Sometimes, calling `reset()` doesn't work the first time and will throw an
# error. In those instances, you can call reset() again and it should work.
# If it still doesn't work, try turning the robot off and on again.
# And if that _still_ doesn't work, take a look at the CAN connection.

# The robot may physically drop when calling this, so move it to a safe position
# or hold it when calling this.
piper_init.reset_arm(
    robot,
    arm_controller=piper_interface.ArmController.POSITION_VELOCITY,
    move_mode=piper_interface.MoveMode.JOINT,
)
piper_init.reset_gripper(robot)

## Move a joint

In [ ]:
joint_angles = robot.get_joint_positions()
joint_angles[-2] -= 0.1
print(f"setting joint angles to {joint_angles}")
robot.command_joint_positions(joint_angles)

# Send Gripper Commands

In [ ]:
# Opens the gripper
robot.command_gripper(
    position=piper_interface.GRIPPER_ANGLE_MAX * 0.9,
    effort=piper_interface.GRIPPER_EFFORT_MAX * 0.5,
)

In [ ]:
# Closes the gripper
robot.command_gripper(
    position=piper_interface.GRIPPER_ANGLE_MAX * 0.1,
    effort=piper_interface.GRIPPER_EFFORT_MAX * 0.5,
)

## Move to special positions

In [ ]:
# TODO(jscholz): Revisit the rest position.
robot.command_joint_positions(piper_control.ArmOrientations.upright.rest_position)

## Cartesian Control

In [ ]:
robot.set_arm_mode(
    move_mode=piper_interface.MoveMode.POSITION,
    arm_controller=piper_interface.ArmController.POSITION_VELOCITY,
)

ee_pose = robot.get_end_effector_pose()
ee_pose[0] += 0.02  # Move X by 5 cm
print(f"Sending updated pose: {ee_pose}")
robot.command_cartesian_position(ee_pose)

## Puppeteering

In [ ]:
robot0 = piper_interface.PiperInterface(can_port="can0")
robot1 = piper_interface.PiperInterface(can_port="can1")

In [ ]:

for rbt in [robot0, robot1]:
    piper_init.reset_arm(
        rbt,
        arm_controller=piper_interface.ArmController.POSITION_VELOCITY,
        move_mode=piper_interface.MoveMode.JOINT,
    )
    piper_init.reset_gripper(rbt)


In [ ]:
robot0.command_joint_positions(piper_control.ArmOrientations.upright.rest_position)

In [ ]:
robot1.command_joint_positions(piper_control.ArmOrientations.upright.rest_position)

In [ ]:
import time
while True:
  joint_angles = robot1.get_joint_positions()
  print(f"{joint_angles=}")
  time.sleep(0.01)
  robot0.command_joint_positions(joint_angles)